<a href="https://colab.research.google.com/github/CRcr0/CUE6156F21/blob/main/MSFT_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install scipy
!pip install scikit-learn
!pip install matplotlib
!pip install pandas

In [2]:
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 8.0 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [14]:
!pip install schedule

In [ ]:
import schedule
import time

def job():
    print("I'm working...")

schedule.every().day.at("07:00").do(job) # set time to run the program

while True:
    schedule.run_pending() # do it
    time.sleep(1)

In [15]:
import yfinance as yf
df = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "MSFT",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "10d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

         # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

[*********************100%***********************]  1 of 1 completed


In [16]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2021-11-19,342.640015,345.100006,342.200012,343.109985,21963400
2021-11-22,344.619995,349.670013,339.549988,339.829987,31031100
2021-11-23,337.049988,339.450012,333.559998,337.679993,30427600
2021-11-24,336.279999,338.160004,333.910004,337.910004,21661300
2021-11-26,334.350006,337.929993,328.119995,329.679993,24217200


In [17]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [18]:
def prepare_data(df,forecast_col,forecast_out,test_size):
    label = df[forecast_col].shift(-forecast_out) #creating new column called label with the last 5 rows are nan
    X = np.array(df[[forecast_col]]) #creating the feature array
    X = preprocessing.scale(X) #processing the feature array
    X_lately = X[-forecast_out:] #creating the column i want to use later in the predicting method
    X = X[:-forecast_out] # X that will contain the training and testing
    label.dropna(inplace=True) #dropping na values
    y = np.array(label)  # assigning Y
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=0) #cross validation

    response = [X_train,X_test , Y_train, Y_test , X_lately]
    return response

In [19]:
forecast_col = 'High'
forecast_out = 1
test_size = 0.2

In [20]:
X_train, X_test, Y_train, Y_test , X_lately =prepare_data(df,forecast_col,forecast_out,test_size)
learner = LinearRegression() #initializing linear regression model

learner.fit(X_train,Y_train) #training the linear regression model

LinearRegression()

In [21]:
score=learner.score(X_test,Y_test)#testing the linear regression model
forecast= learner.predict(X_lately) #set that will contain the forecasted data
response={}#creting json object
#response['test_score']=score
response['forecast_set']=forecast

print(response)

{'forecast_set': array([335.06980253])}


In [22]:
a = forecast
a

array([335.06980253])

In [23]:
b = df.iloc[9]["High"]
b

332.70001220703125

In [24]:
#relative error
c = a-b
d = c/b
print(d)

[0.0071229]
